In [1]:
%matplotlib inline

import glob
import pandas as pd
import yaml
import numpy as np
import cv2
import time

In [2]:
from google.cloud import vision
from google.cloud.vision import types
from google.protobuf import text_format
import io

In [3]:
import os
import sys
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='apikey.json'

In [4]:
path = '.'
prefix = 'E:\\MestradoData\\'
wildcard = '\\*_'
persons = [["p{:03d}".format(i),"{:d}".format(i)] for i in range(45,46)]
#classes = [i for i in range(1,20)]
classes = [1,6,10]
suffix = '_{:d}'
type_file = 'rgb'

In [5]:
persons

[['p045', '45']]

In [6]:
def get_face_rect_default(pclass, person, data, files_list):
    if len(files_list) >= 4:
        sample = [pclass, person]
        for i in range(4):
            sample.append(files_list[i])
            sample.append(face_rect[i][1])
            sample.append(face_rect[i][0])
            sample.append(face_rect[i][1]+face_window)
            sample.append(face_rect[i][0]+face_window)
        #print(sample)
        data.append(sample)

In [7]:
def detect_face_google_vision(file_name):
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()
        
    image = types.Image(content=content)
    
    response = client.face_detection(image=image)
    
    faces = response.face_annotations
    for face in faces:
        vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in face.bounding_poly.vertices])
        print('face bounds: {}'.format(','.join(vertices)))
    
    return response

In [8]:
client = vision.ImageAnnotatorClient()

tic = time.clock()
for c in classes:
    for p in persons:
        for i in range (1,301,30):
            pattern = prefix + p[0] + '\\' + '*' +'_'+ str(c) + '_' + type_file + suffix.format(i) + '.bmp'
            files_list = glob.glob(pattern)
            #print(pattern) 
            print(pattern, len(files_list), files_list)
            
            if len(files_list) == 1:
                try:
                    response = detect_face_google_vision(files_list[0])
                    # Save response to file
                    saved_file = prefix + p[0] + '\\' + p[1] +'_'+ str(c) + '_' + type_file + suffix.format(i) + '.txt'
                    f = open(saved_file, 'w')
                    f.write(text_format.MessageToString(response))
                    f.close()
                except Exception as ex:
                    print(ex)
print(time.clock() - tic)

E:\MestradoData\p045\*_1_rgb_1.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_1.bmp']
face bounds: (142,68),(247,68),(247,190),(142,190)
E:\MestradoData\p045\*_1_rgb_31.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_31.bmp']
face bounds: (138,68),(242,68),(242,189),(138,189)
E:\MestradoData\p045\*_1_rgb_61.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_61.bmp']
face bounds: (141,71),(245,71),(245,192),(141,192)
E:\MestradoData\p045\*_1_rgb_91.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_91.bmp']
face bounds: (143,70),(248,70),(248,193),(143,193)
E:\MestradoData\p045\*_1_rgb_121.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_121.bmp']
face bounds: (143,70),(248,70),(248,193),(143,193)
E:\MestradoData\p045\*_1_rgb_151.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_151.bmp']
face bounds: (143,70),(248,70),(248,193),(143,193)
E:\MestradoData\p045\*_1_rgb_181.bmp 1 ['E:\\MestradoData\\p045\\45_1_rgb_181.bmp']
face bounds: (142,70),(247,70),(247,192),(142,192)
E:\MestradoData\p045\*_1_rgb_211.bmp 1 ['E:\\MestradoData\\p045\\

In [ ]:
# Read response file
f = open('b.txt', 'r')
resp = types.AnnotateImageResponse() # replace with your own message
text_format.Parse(f.read(), resp)
f.close()